In [1]:
!pip install kora -q
from kora import drive
drive.link_nbs()

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import utils

importing Jupyter notebook from /nbs/utils.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
SF_notloaded = True   #set to True after first run once you have your files in the Files folder of Colab 

In [4]:
if not SF_notloaded:
  utils.onceSF()

In [5]:
#!pip install efficientnet

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import efficientnet.keras as efn
from keras import layers


ModuleNotFoundError: ignored

In [ ]:
""" Load dataset """
#Load the dataset which has already been preprocessed
SF_training_set, train_labels, SF_test_set, test_labels, class_names = utils.loadSF40(img_size =  IMG_SIZE)

#Split the trainingset to obtain 10% stratified validation set
train_images, validation_images, train_labels, validation_labels = train_test_split(SF_training_set, train_labels, test_size=0.1, random_state=0, stratify=train_labels)


In [7]:
""" Global variables """
IMG_SIZE = (224,224)
EPOCHS = 50
BATCH_SIZE = 64
NO_TRAIN_IMGS = len(train_labels)
NO_VAL_IMGS = len(validation_labels)


Lengt of train_files:  4000
Action categories (40):
['applauding', 'blowing_bubbles', 'brushing_teeth', 'cleaning_the_floor', 'climbing', 'cooking', 'cutting_trees', 'cutting_vegetables', 'drinking', 'feeding_a_horse', 'fishing', 'fixing_a_bike', 'fixing_a_car', 'gardening', 'holding_an_umbrella', 'jumping', 'looking_through_a_microscope', 'looking_through_a_telescope', 'phoning', 'playing_guitar', 'playing_violin', 'pouring_liquid', 'pushing_a_cart', 'reading', 'riding_a_bike', 'riding_a_horse', 'rowing_a_boat', 'running', 'shooting_an_arrow', 'smoking', 'taking_photos', 'texting_message', 'throwing_frisby', 'using_a_computer', 'walking_the_dog', 'washing_dishes', 'watching_TV', 'waving_hands', 'writing_on_a_board', 'writing_on_a_book']


In [ ]:
"""   Data augmentation and Normalisation """
# data augmentation generator defining the augmentations and data-preprocessing to be made
data_generator = ImageDataGenerator(
        rescale=1.0/255.0, #normalising pixel values to range 0-1
        rotation_range=20, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.5,1.2]  # brightness
        )

#Create iterators to pass to the model during training
train_iterator = data_generator.flow(train_images, train_labels, batch_size=64)
validation_iterator =  data_generator.flow(validation_images, validation_labels, batch_size=64)
test_iterator = data_generator.flow(SF_test_set, test_labels, batch_size=64)


""" This is how we will fit it with the model """
# fit model with generator
# model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
# # evaluate model
# _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)

# https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/ 

""" One-hot encoding """

def onehot_encoding(image, label):
    label = tf.one_hot(label, class_names)
    return image, label


train_images = train_images.map(
    onehot_encoding, num_parallel_calls=tf.data.AUTOTUNE)

train_images = train_images.batch(batch_size=BATCH_SIZE, drop_remainder=True)
train_images = train_images.prefetch(tf.data.AUTOTUNE)

SF_test_set = SF_test_set.map(onehot_encoding)
SF_test_set = SF_test_set.batch(batch_size=BATCH_SIZE, drop_remainder=True)


In [ ]:
""" Tutorials used to build model"""
#https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
#https://towardsdatascience.com/an-in-depth-efficientnet-tutorial-using-tensorflow-how-to-use-efficientnet-on-a-custom-dataset-1cab0997f65c
#https://www.kaggle.com/code/arjunrao2000/beginners-guide-efficientnet-with-keras/notebook

""" Cyclical learning rate example"""
#https://pyimagesearch.com/2019/07/29/cyclical-learning-rates-with-keras-and-deep-learning/

In [ ]:
""" Build & Train Stanford40 Model EfficientNet"""

# Define the input and output layers of the model 
inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

outputs = efn.EfficientNetB0(include_top=True, weights=None, classes=class_names) #Include top set to true as we are not using pre-trained weights but training from scratch

# Initialise the model. Compule and show summary. 
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

#Fit the model with the data augmentation and normalisation iterators
history = model.fit_generator(train_iterator, epochs = EPOCHS, steps_per_epoch = NO_TRAIN_IMGS//BATCH_SIZE, 
                              validation_data = validation_iterator, validation_steps=NO_VAL_IMGS//BATCH_SIZE, 
                              verbose=1, use_multiprocessing=True, workers=4)

#Save weights as we need these for transfer learning 
model.save_weights("model_weights.h5")


In [ ]:
TL_model = 


TL_model.load_weights("model_weights.h5")